#  分子指纹

分子可以用许多方式表示。本教程介绍了一种被称为"分子指纹"的表示方法，这是一种非常简单的表示方法，通常对类似药物的小分子很有效。

## Colab

本教程和目录中的其余部分都是在 Google colab 中完成。如果您想在 colab 中打开此笔记本，您可以点击以下链接。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BioGavin/deepchem/blob/master/examples/tutorials-zh/Molecular_Fingerprints.ipynb)



In [ ]:
!pip install --pre deepchem

我们现在可以导入 deepchem 包来玩耍。

In [1]:
import deepchem as dc
dc.__version__

'2.4.0-rc1.dev'

# 什么是分子指纹？

深度学习模型几乎总是以数组作为其输入，如果我们想用它来处理分子，我们需要以某种方式将每个分子表示为一个或多个数组。

大多数（但不是所有）模型要求其输入有一个固定的尺寸，这对分子来说是个挑战，因为不同的分子有不同的原子数量，如果我们想使用这些类型的模型，我们需要以某种方式用固定大小的数组来表示可变大小的分子。

指纹就是为了解决这些问题而设计的。指纹是一个固定长度的数组，不同的元素表示分子中存在的不同特征。如果两个分子有相似的指纹，这表明它们含有许多相同的特征，因此很可能具有相似的化学性质。

DeepChem 支持一种特殊类型的指纹，称为"扩展连接性指纹"，简称为"ECFP"。它们有时也被称为"环形指纹"。ECFP 算法首先只根据原子的直接属性和键进行分类，每个独特的模式都是一个特征，例如，与两个氢和两个重原子结合的碳原子就是一个特征，对于包含该特征的任何分子，指纹的一个特定元素被设置为1，然后，它通过查看更大的圆形邻域来迭代识别新的特征，一个特定的特征与其他两个特定的特征结合在一起，成为一个更高层次的特征，任何包含该特征的分子都会被设置相应的元素。这将持续一个固定的迭代次数，最常见的是两次。

让我们来看看一个用 ECFP 处理过的数据集。

In [2]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP')
train_dataset, valid_dataset, test_dataset = datasets
print(train_dataset)

<DiskDataset X.shape: (6264, 1024), y.shape: (6264, 12), w.shape: (6264, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>


特征数组 `X` 的形状为（6264，1024）。这意味着在训练集中有 6264 个样本，每一个样本都由一个长度为 1024 的指纹来代表，同时注意到标签数组 `y` 的形状为（6264， 12），这是一个多任务数据集。Tox21 包含关于分子毒性的信息，12种不同的检测方法被用来寻找毒性的迹象。该数据集每个样本记录了所有12个检测的结果，每个都是一个不同的任务。

我们也来看看权重数组。

In [3]:
train_dataset.w

array([[1.0433141624730409, 1.0369942196531792, 8.53921568627451, ...,
        1.060388945752303, 1.1895710249165168, 1.0700990099009902],
       [1.0433141624730409, 1.0369942196531792, 1.1326397919375812, ...,
        0.0, 1.1895710249165168, 1.0700990099009902],
       [0.0, 0.0, 0.0, ..., 1.060388945752303, 0.0, 0.0],
       ...,
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [1.0433141624730409, 1.0369942196531792, 8.53921568627451, ...,
        1.060388945752303, 0.0, 0.0],
       [1.0433141624730409, 1.0369942196531792, 1.1326397919375812, ...,
        1.060388945752303, 1.1895710249165168, 1.0700990099009902]],
      dtype=object)

注意有些元素是 0，这样的权重被用来表示缺失的数据。实际上并不是所有的检测都是在每个分子上进行的。将一个样本或样本/任务对的权重设置为0，会使其在拟合和评估过程中被忽略，单它对损失函数或其他指标没有影响。

这样做是为了平衡每个任务中正负样本的总体权重。在训练模型时，我们希望 12 个任务中的每一个都能做出同样的贡献，在每个任务中，我们希望对正负样本给予同等的权重，否则，模型可能只知道大多数训练样本是无毒的，因此会偏向于识别其他分子为无毒的。

# 对指纹进行模型训练

让我们来训练一个模型。在前面的教程中，我们使用 `GraphConvModel`，这是一个相当复杂的架构，需要一组复杂的输入。因为指纹非常简单，只是一个固定长度的数组，所以我们可以使用一个更简单的模型。

In [4]:
model = dc.models.MultitaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000])

`MultitaskClassifier` 是一个简单的全连接层的堆。在这个例子中，我们告诉模型使用一个宽度为 1000 的单隐藏层。我们还告诉模型每个输入将有 1024 个特征，并且它应该对 12 个不同的任务做出预测。

为什么不为每个任务训练一个单独的模型呢？我们可以这样做，但事实证明，为多个任务训练一个单一的模型往往效果更好。我们将在后面的教程中看到一个这样的例子。

让我们来训练和评估这个模型。

In [5]:
import numpy as np

model.fit(train_dataset, nb_epoch=10)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('training set score:', model.evaluate(train_dataset, [metric], transformers))
print('test set score:', model.evaluate(test_dataset, [metric], transformers))

training set score: {'roc_auc_score': 0.9550063590563469}
test set score: {'roc_auc_score': 0.7781819573695475}


对于这样一个简单的模型和特征化来说，模型的结果表现不错。更复杂的模型在这个数据集上的表现稍好，但也不会非常好。

# 恭喜！是时候加入社区了！

恭喜您完成本教程笔记本！如果您喜欢本教程并希望继续使用 DeepChem，我们鼓励您完成本系列的其余教程。您还可以通过以下方式帮助 DeepChem 社区：

## 在 [GitHub](https://github.com/deepchem/deepchem) 上为 DeepChem 点亮小星星
这有助于大家建立对 DeepChem 项目和我们正在尝试构建的开源药物发现工具的共识。

## 加入 DeepChem Gitter
DeepChem [Gitter](https://gitter.im/deepchem/Lobby) 聚集了许多对生命科学深度学习感兴趣的科学家、开发人员和爱好者，欢迎加入！